<p style="text-align: center;"><span style="color: #ff0000;"><strong><span style="font-size: x-large;">
    ANEXO 6: ALGORITMOS PKE</span></strong></span></p>

<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Realizado por:</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Gabriel Vacaro Goytia</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Ignacio Warleta Murcia</span></strong></span></p>

Este notebook contiene una implementación de los algoritmos fundamentales del esquema de cifrado post–cuántico Kyber, en particular el KyberKEM (Key Encapsulation Mechanism). Kyber es un sistema criptográfico basado en el problema de Redes Lattice y es uno de los candidatos más destacados para ser parte del estándar de criptografía post-cuántica propuesto por el NIST.

Organizamos el anexo según el siguiente índice:

# Índice

1. [Introducción](#1.-Introducción)
2. [Configuración previa](#2.-Configuracion-previa)
3. [Algoritmo de generacion de claves KEM](#3.-Algoritmo-de-generacion-de-claves-KEM)
4. [Algoritmo de encapsulado](#4.-Algoritmo-de-encapsulado)
5. [Algoritmo de desencapsulado](#5.-Algoritmo-de-desencapsulado)
6. [Ejemplo de uso](#6.-Ejemplo-de-uso)

---
# 1. Introducción






Este notebook cuenta con los 3 algoritmos relativos al KEM de Kyber–KEM:

- Generación de claves (Key Generation) <br>
- Encapsulado (Encapsulation) <br>
- Desencapsulado (Decapsulation) <br>

El propósito de esta implementación es comprender los detalles detrás de cada fase del esquema Kyber–KEM de manera didáctica, con especial énfasis en los aspectos técnicos que permiten asegurar la privacidad y la integridad de la comunicación en un entorno potencialmente afectado por computadoras cuánticas. <br>


---
# 2. Configuracion previa

A continuación, se muestra la configuración previa a ejecutar y los parámetros a definir. En este caso hemos optado por un primo $q$ relativamente pequeño, 743, dado su caracter didáctico, ya que, de esta manera se puede apreciar ligeramente el error que se produce al descifrar el mensaje. Por supuesto, el usuario es libre de cambiar este primo a su gusto y comprobar por su cuenta como se maneja el error. Además haremos uso de los algoritmos ya vistos relativos al PKE como son la generación de claves, el cifrado y descifrado. Cabe decir que se han realizado modificaciones al método implementado de cifrado para que sea capaz de soportar las semillas de mayor tamaño con las que trabajamos en encapsulación.

In [33]:
#MODULOS A IMPORTAR
import numpy as np
import hashlib
import secrets

In [34]:
# Parámetros básicos
q = 743  # Un número primo pequeño típico en Kyber podría ser 3329
k = 3  # Tamaño del vector/matriz (varía según los estándares Kyber-512, 768, 1024)
mu_1 = 0.0  # Parámetro para la distribución de error más controlado (menor desviación estándar)

In [35]:
# Algoritmos PKE

# Funciones auxiliares

def generate_seed(n=16):
    
    """
    Generación de semilla aleatoria de n bytes 
     
    Parámetros:
    - n: número de bytes (16 por defecto)
    
    Retorna:
    - Semilla aleatorio de n bytes.
    """
    return secrets.token_bytes(n)

def generate_matrix_A(seed, k, q):
    """
    Esta función genera una matriz aleatoria k × k con valores en el rango 
    [0,q), utilizando una semilla dada para asegurar la reproducibilidad de los números aleatorios.
     
    Parámetros:
    - seed: Un valor en bytes que se usa para inicializar la semilla del generador de números aleatorios.
    - k: Dimensión de la matriz cuadrada que se va a generar.
    - q: módulo sobre el que se trabaja.
    
    Retorna:
    - Matriz aleatorio kxk con valores en [0,q)
    """
    np.random.seed(int.from_bytes(seed, "big") % (2**32)) # Convertir semilla a entero usando big-endian limitado a 32 bits
    return np.random.randint(0, q, size=(k, k)) 

def sample_error(mu, k, q, seed=None):
    """
    Esta función genera un vector de errores siguiendo una distribución normal centrada en 0 
    con desviación estándar mu en módulo q. Si no se introduce semilla, se utilizará 
    el estado actual del generador aleatorio de NumPy, permitiendo que los valores 
    generados varíen en cada ejecución. 
     
    Parámetros:
    - mu: Desviación estándar de la distribución normal.
    - k: Número de filas del vector (dimensión del error).
    - q: Módulo sobre el cual se trabaja.
    - seed: Valor opcional para fijar la semilla del generador aleatorio.
    
    Retorna:
    - Vector k dimensional de valores aleatorios en el rango [0,q).
    """
    if seed is not None:
        state = np.random.get_state()  # Guarda el estado actual del generador
        if isinstance(seed, bytes):  # Si es de tipo Bytes:
            seed = int.from_bytes(seed, "big") % (2**32) # Refundicion a entero en rango de 32 bits.
        np.random.seed(seed)  # Fija la semilla para reproducibilidad

    error = np.round(np.random.normal(0, mu, size=(k, 1))).astype(int) % q  # Genera el vector de errores

    if seed is not None:
        np.random.set_state(state)  # Restaura el estado original del generador

    return error

# Algoritmo 1: Generación de Claves G'
def key_generation():
    """
    Generación de claves pública y secreta para un esquema basado en LWE; relativo al PKE.
    
    Retorna:
    - Clave pública (b, seed_A): Vector b y semilla para regenerar la matriz A.
    - Clave secreta s: Vector secreto utilizado para descifrar.
    """
    seed_A = generate_seed() # Se genera una semilla aleatoria para construir la matriz A de forma determinista
    A = generate_matrix_A(seed_A, k, q) # Se genera la matriz A de dimensión k x k en módulo q
    s = sample_error(mu_1, k, q)  # Se genera el vector secreto s con distribución gaussiana
    e = sample_error(mu_1, k, q)  # Se genera el vector error la misma distribución
    b = (A @ s + e) % q   # Se calcula el vector b como b = (A * s + e) mod q
    return (b, seed_A), s

# Algoritmo 2: Cifrado modificado para usar semilla
def encrypt(pk, m, seed=None):
    """
    Esta función implementa el algoritmo de cifrado del PKE modificado con el uso opcional 
    de una semilla para garantizar la reproducibilidad de los valores aleatorios 
    generados.
    
    Parámetros:
    - pk: Clave pública (b, seed_A), donde:
        - b: Vector b generado en la fase de generación de claves.
        - seed_A: Semilla usada para reconstruir la matriz A.
    - m: Mensaje a cifrar (normalmente representado como un número en ℤ_q).
    - seed: Valor opcional que, si se proporciona, permite generar errores y aleatoriedad 
      de forma determinista mediante el uso de una función hash.

    Retorna:
    - (u, v): Par de valores cifrados.
    - r: Vector de aleatorización utilizado en el cifrado.
    - e_1: Vector de error agregado a u.
    - e_2: Valor de error agregado a v.
    """
    b, seed_A = pk # valores de la clave pública
    if seed:     # Si se proporciona una semilla, se usa SHA3-256 para derivar valores deterministas
        hash_output = hashlib.sha3_256(seed).digest() # Hash de la semilla
        r_seed = hash_output[:4]  # Usar solo 4 bytes (32 bits)
        e1_seed = hash_output[4:8]
        e2_seed = hash_output[8:12]
        # Se generan los vectores de aleatorización y errores de forma determinista
        r = sample_error(mu_1, k, q, int.from_bytes(r_seed, 'big'))
        e_1 = sample_error(mu_1, k, q, int.from_bytes(e1_seed, 'big'))
        e_2 = sample_error(mu_1, k, q, int.from_bytes(e2_seed, 'big'))
    else:
        # Si no hay semilla no será reproducible
        r = sample_error(mu_1, k, q)
        e_1 = sample_error(mu_1, k, q)
        e_2 = sample_error(mu_1, k, q)
    
    A = generate_matrix_A(seed_A, k, q)     
    u = (A.T @ r + e_1) % q  # Se calcula u = (Aᵀ * r + e₁) mod q
    v = (b.T @ r + e_2 + m) % q # Se calcula v = (bᵀ * r + e₂ + m) mod q
    return (u, v), r, e_1, e_2

def decrypt(sk, c):
    """
    implementa el algoritmo de descifrado basado en LWE; relativo al PKE, recuperando el mensaje original 
    a partir del par cifrado (u, v) y la clave secreta s.

    Parámetros:
    - sk: Clave secreta utilizada en la generación de claves.
    - c: Texto cifrado (u, v), donde:
      - u: Vector generado durante el cifrado.
      - v: Valor resultante que contiene el mensaje más ruido.

    Retorna:
    - El mensaje descifrado en el dominio ℤ_q.
    """
    u, v = c  
    s = sk  
    # Se recupera el mensaje m utilizando la relación v - sᵀ * u mod q
    return (v - (s.T @ u)) % q  

---
# 3. Algoritmo de generacion de claves KEM

1. **Generación del vector $z$:**  
   Primeramente, generamos un vector $z$ de 256 bits aleatorios. Este valor se utiliza como «entropía» o «salto aleatorio» y es esencial para mantener la seguridad del sistema. Se usa en la clave secreta final $sk$, y la elección de 256 bits proporciona suficiente aleatoriedad para asegurar que la clave secreta sea impredecible.

2. **Llamada al algoritmo de generación de claves $G'$:**  
   En el siguiente paso, llamamos al algoritmo de generación de claves $G'$ descrito anteriormente. De este algoritmo, obtenemos la clave pública $pk$ y la clave secreta intermedia $sk'$, que solo se utilizará para generar la nueva clave secreta final $sk$.

3. **Construcción de la clave secreta final $sk$:**  
   En este paso, construimos la clave secreta final $sk$ concatenando varios componentes:  
   - La clave secreta intermedia $sk'$.  
   - La clave pública $pk$.  
   - El resultado de aplicar una función hash $H$ sobre la clave pública, $H(pk)$.  
   El propósito de aplicar la función hash sobre la clave pública es incluir un resumen de la misma en la clave secreta. Esto mejora la seguridad, ya que asegura que la clave secreta dependa de la clave pública, lo que ayuda a evitar ciertos tipos de ataques.

4. **Concatenación final y devolución de claves:**  
   Finalmente, concatenamos el vector $z$ y se devuelve la clave pública $pk$ junto con la clave secreta final $sk$.


In [36]:
# Algoritmo de Generación de Claves KEM
def key_gen_G():
    """
    Esta función implementa la generación de claves para el estandar Kyber-KEM.
    Utiliza un esquema basado en LWE y genera tanto la clave pública como la clave secreta.
    
    Retorna:
    - pk: Clave pública generada.
    - sk: Clave secreta generada.
    """
    z = secrets.token_bytes(32) # Generación del vector z de 256 bits. 
    pk, sk_prime = key_generation()  # Generación de la clave pública y la clave secreta intermedia.
    b, seed_A = pk
    sk_prime_bytes = sk_prime.tobytes() #Convierte sk_prime en una secuencia de bytes 
    pk_bytes = b.tobytes() + seed_A     # Se convierte el vector b y la semilla seed_A en bytes y se concatenan
    H_pk = hashlib.sha3_256(pk_bytes).digest()   #Calcula el hash SHA3–256 y devuelve una secuencia de bytes
    sk = sk_prime_bytes + pk_bytes + H_pk + z     # La clave secreta final se construye concatenando sk_prime, pk_bytes, H_pk y z
    return pk, sk

---
# 4. Algoritmo de encapsulado

1. **Generación del valor $m'$:**  
   El primer paso del algoritmo de encapsulado consiste en generar un valor $m'$ de 256 bits aleatorios. Luego, en el siguiente paso, se produce un valor $m$ al aplicarle a $m'$ una función hash $H$.  
   El propósito de este paso es generar una «clave derivada» a partir de los 256 bits aleatorios, pero con una forma adecuada para el cifrado, es decir, con una longitud estándar, lo cual se consigue gracias a la función hash $H$.

2. **Construcción de la clave compartida $K ̅$ y valor $r$:**  
   En el tercer paso, construimos una clave $K ̅$ y un valor $r$ concatenando $m$ y el hash de la clave pública, $H(pk)$.  
   - $K ̅$ es una clave compartida que se utilizará para la comunicación posterior.  
   - La concatenación de $m$ y $H(pk)$ garantiza que $K ̅$ dependa tanto del valor aleatorio inicial como de la clave pública del destinatario.  
   Esto asegura que el proceso de cifrado sea seguro y dependiente de la clave pública.

3. **Cifrado del mensaje ($c$):**  
   En el siguiente paso, se realiza el cifrado utilizando la clave pública y los valores $m$ y $r$ como parámetros de entrada.  
   El resultado de este paso es el mensaje cifrado $c$.

4. **Derivación de la clave compartida final $K$:**  
   Seguidamente, se realiza una función de derivación de claves (KDF, Key Derivation Function) para generar la clave compartida final $K$.  
   - Primero, se concatenan la clave $K ̅$ generada en el paso 2 y el hash de $c$.  
   - La función KDF toma esta concatenación y produce la clave derivada final $K$.  
   El uso de KDF asegura que la clave derivada final $K$ tenga una longitud adecuada y sea segura para ser utilizada como una clave secreta compartida entre el emisor y el receptor.

5. **Devolución de los resultados:**  
   Finalmente, el algoritmo devuelve el mensaje cifrado $c$ y la clave compartida $K$.


In [37]:
# Algoritmo de Encapsulado KEM
def encapsulate(pk):
    """
    Esta función implementa el mecanismo de encapsulación de clave relativo a Kyber-KEM, 
    generando una clave compartida segura a partir de la clave pública del receptor.

    Parámetros:
    - pk: Clave pública del receptor, utilizada para cifrar el mensaje.

    Retorna:
    - c: Texto cifrado resultante.
    - K: Clave compartida derivada.
    """
    m_prime = secrets.token_bytes(32)     # Se genera un valor aleatorio m' de 256 bits (32 bytes)
    m = hashlib.sha3_256(m_prime).digest() # Se aplica SHA3-256 a m' para obtener m
    b, seed_A = pk   # Se obtiene la clave pública (b, seed_A)
    pk_bytes = b.tobytes() + seed_A  # Se convierte la clave pública en una secuencia de bytes
    H_pk = hashlib.sha3_256(pk_bytes).digest()    # Se calcula el hash SHA3-256 de la clave pública
    m_Hpk = m + H_pk # Se concatena m con H(pk) para obtener una clave compartida intermedia K_bar
    K_bar = m_Hpk[:32] #la clave compartida la componen los 32 primeros bytes
    r_seed = m_Hpk[32:36]  # 4 bytes para la semilla

    #Convertimos fragmentos de una secuencia de bytes m en una lista de enteros (m_ints) aplicandotelas un módulo q a cada uno
    m_ints = [int.from_bytes(m[i*4:(i+1)*4], 'big') % q for i in range(k)] 
    m_vector = np.array(m_ints, dtype=int).reshape((k, 1)) #Convertimos la lista en un vector columna
    
    c, _, _, _ = encrypt(pk, m_vector, r_seed) # Se cifra el mensaje utilizando la clave pública y la semilla 
    u, v = c
    c_bytes = u.tobytes() + v.tobytes() # Se convierte el texto cifrado en una secuencia de bytes
    H_c = hashlib.sha3_256(c_bytes).digest()   # Se calcula el hash SHA3-256 del mensaje cifrado c
    K = hashlib.sha3_256(K_bar + H_c).digest() # Se aplica SHA3-256 a la concatenación de K_bar y H_c para obtener la clave final K
    
    print("🔎 Depuración en encapsulate:")
    print(f"🟢 Clave pública utilizada: {pk}")
    print(f"🟡 Valor aleatorio m': {m_prime.hex()}")
    print(f"🟡 Hash de m' (m): {m.hex()}")
    print(f"🟣 m_vector: {m_vector}")
    print(f"🟡 Hash de clave pública (H_pk): {H_pk.hex()}")
    print(f"🟡 K_bar (32 bytes de m + H_pk): {K_bar.hex()}")
    print(f"🟡 Semilla r_seed (4 bytes): {r_seed.hex()}")
    print(f"🟣 Mensaje cifrado (c): {(c[0].tobytes() + c[1].tobytes()).hex()}")
    print(f"🟡 Hash del mensaje cifrado (H_c): {H_c.hex()}")
    print(f"🔐 Clave final encapsulada K: {K.hex()}\n")

    return c, K

---
# 5. Algoritmo de desencapsulado

El objetivo del desencapsulado es que el receptor recupere la clave compartida $K$ a partir de un mensaje cifrado $c$. Para ello, el receptor usa la clave secreta $sk$ y realiza una serie de pasos de verificación para asegurarse de que el mensaje cifrado no ha sido alterado.

1. **Cálculo del valor $h$:**  
   El receptor comienza calculando un valor $h$ a partir de su clave secreta $sk$.  
   Este valor $h$ se deriva utilizando parámetros específicos como $k$ (tamaño de la matriz) y $n$ (número de dimensiones), además de una constante $+32$. Este valor $h$ se utilizará en el proceso de verificación del mensaje cifrado.

2. **Cálculo del valor $z$:**  
   Luego, se calcula el segundo valor, $z$, que también es una combinación de la clave secreta $sk$ y los parámetros $k$ y $n$, pero con la constante $+64$.  
   Este valor $z$ es un valor intermedio que se utilizará en caso de que el mensaje cifrado haya sido alterado.

3. **Descifrado del mensaje:**  
   A continuación, el receptor utiliza su clave secreta $sk$ para descifrar el mensaje $c$ utilizando el algoritmo de descifrado $D(sk,c)$ (Algoritmo 3), obteniendo el mensaje $m ̃$.

4. **Generación de la clave compartida $K ̅$ y valor $r'$:**  
   Con el mensaje recuperado $m ̃$ y el valor $h$ calculado en el primer paso, el receptor genera una clave compartida $K ̅$ y un valor $r'$ mediante la función hash $G(m ̃||h)$.  
   - $K ̅$ es la clave compartida generada a partir de $m ̃$ y $h$.  
   - $r'$ es un valor adicional utilizado en el siguiente paso para garantizar la integridad.  
   La función hash $G$ asegura que el proceso sea seguro y único. Debido a que $G$ es una función hash criptográfica, tanto $K ̅$ como $r'$ son determinísticos pero difíciles de predecir sin conocer $m ̃$ y $h$.

5. **Generación del mensaje cifrado $c'$:**  
   Después de obtener $K ̅$ y $r'$, el receptor genera un nuevo mensaje cifrado $c'$ utilizando la función de cifrado $E(pk, m ̃, r')$.

6. **Comparación de los mensajes cifrados:**  
   El receptor compara el mensaje cifrado $c'$ con el mensaje cifrado $c$ que recibió.  
   - Si $c'$ es igual a $c$, significa que el proceso fue correcto y que $K ̅$ es la clave compartida correcta.  
   - Si la verificación falla, el receptor utiliza el valor alternativo $z$ para generar la clave $K$ de forma diferente. Esta clave será totalmente inútil para un atacante.

7. **Generación de la clave final $K$:**  
   A continuación, el receptor genera la clave final $K$ mediante la función de derivación de claves (KDF).  
   - Se concatena la clave derivada $K ̅$ con el valor hash $H(c)$ de $c$, lo que ayuda a asegurar la integridad y autenticidad del mensaje cifrado.

Si la verificación fue exitosa, el receptor tendrá la clave final $K$, que se utilizará para la comunicación segura.


In [40]:
# Algoritmo de Desencapsulado KEM
def decapsulate(sk, c):
    """
    Esta función implementa el mecanismo de desencapsulación de Kyber-KEM, permitiendo al receptor 
    recuperar la clave compartida K a partir del mensaje cifrado c y su clave secreta sk.

    Parámetros:
    - sk: Clave secreta del receptor, utilizada para descifrar el mensaje cifrado.
    - c: Texto cifrado recibido.

    Retorna:
    - K: Clave compartida derivada tras la verificación de integridad del mensaje cifrado.
    """
    
    sk_prime_len = 4 * k # Tamaño de la clave secreta sk'
    pk_len = 4*k + 16    # Tamaño de la clave pública en bytes
    h_start = sk_prime_len + pk_len   # Posición de inicio de h dentro de sk
    h = sk[h_start:h_start+32]        # Se obtiene h (32 bytes)
    z = sk[h_start+32:h_start+64]     # Se obtiene z (32 bytes)
    
    #Toma los primeros bytes de sk y los interpreta como enteros de 32 bits (.int32)
    #Luego los reorganiza en un vector columna Kx1
    s = np.frombuffer(sk[:sk_prime_len], dtype=np.int32).reshape((k,1)) 
    
    # Se reconstruye la clave pública pk a partir de los siguientes bytes de sk.
    pk = (np.frombuffer(sk[sk_prime_len:sk_prime_len+4*k], dtype=np.int32).reshape((k,1)), 
          sk[sk_prime_len+4*k:sk_prime_len+pk_len])

    # Se descifra el mensaje cifrado c utilizando la clave secreta sk'
    m_tilde = decrypt(s, c)
    m_tilde_bytes = m_tilde.tobytes()
    m_tilde_ints = [int.from_bytes(m_tilde[i*4:(i+1)*4], 'big') % q for i in range(k)] 
    m_tilde_vector = np.array(m_tilde_ints, dtype=int).reshape((k, 1)) #Convertimos la lista en un vector columna    

    # Se genera la clave compartida intermedia K_bar' y el valor r' a partir de m~ y h
    m_tilde_h = m_tilde_bytes + h # Se concatena m~ con h
    G_output = hashlib.sha3_256(m_tilde_h).digest()   # Se calcula el hash SHA3-256
    K_bar_prime = G_output[:32]    # Los primeros 32 bytes corresponden a la clave compartida K_bar'
    r_prime_seed = G_output[32:]   # Los siguientes bytes corresponden a la semilla r'

    # Se vuelve a cifrar el mensaje m~ con la clave pública pk y la semilla r'
    c_prime, _, _, _ = encrypt(pk, m_tilde, r_prime_seed)
    u, v = c   # Se extraen las partes del mensaje cifrado original
    c_bytes = u.tobytes() + v.tobytes()  # Se convierte el mensaje cifrado original en bytes
    u_prime, v_prime = c_prime  # Se extraen las partes del mensaje cifrado regenerado
    c_prime_bytes = u_prime.tobytes() + v_prime.tobytes() # Se convierte en bytes
    
    H_c = hashlib.sha3_256(c_bytes).digest() # Se calcula el hash del mensaje cifrado original
    
    print("🔎 Depuración en decapsulate:")
    print(f"🟢 Clave pública reconstruida: {pk}")
    print(f"🟡 Valor h: {h.hex()}")
    print(f"🟡 Valor z: {z.hex()}")
    print(f"🟣 m_tilde_vector: {m_tilde_vector}")
    print(f"📩 Mensaje descifrado (m_tilde): {m_tilde.tobytes().hex()}")
    print(f"🟡 r_prime_seed: {r_prime_seed}")
    print(f"🟡 Hash de m_tilde + h: {hashlib.sha3_256(m_tilde_bytes + h).digest().hex()}")
    print(f"🟡 K_bar_prime (primeros 32 bytes del hash): {K_bar_prime.hex()}")
    print(f"🟣 Mensaje cifrado original (c): {(c[0].tobytes() + c[1].tobytes()).hex()}")
    print(f"🟣 Mensaje cifrado regenerado (c_prime): {(c_prime[0].tobytes() + c_prime[1].tobytes()).hex()}")
    print(f"🔍 ¿c == c_prime? {'✅ Sí' if c_bytes == c_prime_bytes else '❌ No'}")
    
    if c_bytes == c_prime_bytes:
        print(f"🔐 Clave final desencapsulada (usando K_bar_prime): {hashlib.sha3_256(K_bar_prime + H_c).digest().hex()}")
    else:
        print(f"🔐 Clave final desencapsulada (usando z): {hashlib.sha3_256(z + H_c).digest().hex()}")


    if c_bytes == c_prime_bytes: # Se compara el mensaje cifrado regenerado con el original
        return hashlib.sha3_256(K_bar_prime + H_c).digest()         # Si coinciden, se usa K_bar' para generar la clave final K

    else:
        return hashlib.sha3_256(z + H_c).digest()         # Si no coinciden, se usa z en su lugar para generar la clave final K


---
# 6. Ejemplo de uso

A continuación se muestra un ejemplo de uso utilizando los tres algoritmos:

In [42]:
"""
El problema parece estar en que el mensaje m y m_tilde no coinciden, 
por tanto la clave compartida k_bar no coincide y hace efecto bola y no coinciden las claves finales.
""" 
def print_header(title):
    print("\n" + "═" * 50)
    print(f"{title.center(50)}")
    print("═" * 50 + "\n")

# 1. Generar claves KEM
print_header("🔑 GENERACIÓN DE CLAVES KEM 🔑")
pk, sk = key_gen_G()

print("📜 Clave Pública (pk):")
print(f"🟢 {pk}\n")
print("🔐 Clave Secreta (sk):")
print(f"🟡 {sk}\n")

# 2. Encapsulación de clave
print_header("🔒 ENCAPSULACIÓN DE CLAVE 🔒")
ciphertext, K_encap = encapsulate(pk)
print("📦 Clave encapsulada:")
print(f"🟣 {K_encap.hex()}\n")

# 3. Desencapsulación de clave
print_header("🔓 DESENCAPSULACIÓN DE CLAVE 🔓")
K_decap = decapsulate(sk, ciphertext)
print("📦 Clave desencapsulada:")
print(f"🟡 {K_decap.hex()}\n")

# 4. Comparación de claves
print_header("📊 COMPARACIÓN DE CLAVES 📊")
status = "✅ Coinciden" if K_encap == K_decap else "❌ No coinciden"
print(f"Resultado: {status}\n")



══════════════════════════════════════════════════
           🔑 GENERACIÓN DE CLAVES KEM 🔑           
══════════════════════════════════════════════════

📜 Clave Pública (pk):
🟢 (array([[0],
       [0],
       [0]]), b'\xac<F\xfc\x81\x0f\xa0\xaei\x81{\xaa\xd6\xc7\xf5n')

🔐 Clave Secreta (sk):
🟡 b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xac<F\xfc\x81\x0f\xa0\xaei\x81{\xaa\xd6\xc7\xf5n\xa0\xdfd\xa0\xf5\xff#X\xb2\x88/6\xd0@\xb2>g\xc7_\xa5\x87\xff\xff\xcag\x9c\xa5@5'\xe8\xd3>x&\x9a{\xa7\xe1\x89\xd3\xdf\xde<\xd1$f\x01\x8e\x04\x8e2z\x88\xbf\x01O\xc7I,\x89\x19\xdf\x87"


══════════════════════════════════════════════════
            🔒 ENCAPSULACIÓN DE CLAVE 🔒            
══════════════════════════════════════════════════

🔎 Depuración en encapsulate:
🟢 Clave pública utilizada: (array([[0],
       [0],
       [0]]), b'\xac<F\xfc\x81\x0f\xa0\x

---
# 7. Ejemplo de uso didáctico

Dada la complejidad de cálculo del ejemplo anterior, se han realizado algunas modificaciones extra en los métodos de cifrado, descifrado encapsulado y densecapsulado, así como en la elección de parámetros, con el fin de ver que los cálculos son correctos y que realmente los algoritmos funcionan y cumplen su propósito.

In [45]:
# PARÁMETROS REDUCIDOS
q = 127       # Modulo más pequeño (original 3329)
k = 2         # Dimensión reducida (original 3)
mu_1 = 0.5    # Menor dispersión de errores
SEED_BYTES = 8  # Semillas más cortas (original 16)

In [47]:
# Funciones ajustadas
def generate_seed():
    return np.random.bytes(SEED_BYTES)

def generate_matrix_A(seed, k, q):
    np.random.seed(int.from_bytes(seed[:4], "big"))  # Usa solo 4 bytes
    return np.random.randint(0, q, size=(k, k))

def sample_error(mu, k, q, seed=None):
    if seed is not None:
        seed = seed % (2**32) #Módulo 2^32 
        np.random.seed(seed)
    return np.random.randint(-1, 2, size=(k, 1)) % q  # Error uniforme simple

######## Generación de claves simplificada ########
def key_generation():
    seed_A = generate_seed()
    A = generate_matrix_A(seed_A, k, q)
    s = sample_error(mu_1, k, q)
    e = sample_error(mu_1, k, q)
    b = (A @ s + e) % q
    return (b, seed_A), s

######## Cifrado optimizado ########
def encrypt(pk, m, seed=None):
    b, seed_A = pk
    if seed:
        r_seed = int.from_bytes(seed[:4], 'big') % (2**32)
        r = sample_error(mu_1, k, q, r_seed)
    else:
        r = sample_error(mu_1, k, q)
    
    A = generate_matrix_A(seed_A, k, q)
    u = (A.T @ r) % q
    v = (b.T @ r + m) % q  # Sin error adicional
    return (u, v)

######## Descifrado ########
def decrypt(sk, c):
    u, v = c
    return (v - (sk.T @ u)) % q

######## KEM simplificado ########
def key_gen_G():
    pk, sk = key_generation()
    return pk, sk.tobytes()  # Clave secreta simple

def encapsulate(pk):
    m = np.random.randint(0, q, size=(k, 1))
    c = encrypt(pk, m)
    K = hashlib.shake_128(m.tobytes()).digest(16)  # KDF ligero
    return c, K

def decapsulate(sk_bytes, c):
    sk = np.frombuffer(sk_bytes, dtype=int).reshape((k, 1))
    m = decrypt(sk, c)
    return hashlib.shake_128(m.tobytes()).digest(16)

######## Ejemplo de uso ########
if __name__ == "__main__":
# 1. Generar claves KEM
    print_header("🔑 GENERACIÓN DE CLAVES KEM 🔑")
    pk, sk = key_gen_G()
    print(f"📏 Tamaño clave pública: {len(pk[0].tobytes()) + len(pk[1])} bytes")
    print(f"📏 Tamaño clave secreta: {len(sk)} bytes\n")
    
    print("📜 Clave Pública (pk):")
    print(f"🟢 {pk}\n")
    print("🔐 Clave Secreta (sk):")
    print(f"🟡 {sk}\n")

    # 2. Encapsulación de clave
    print_header("🔒 ENCAPSULACIÓN DE CLAVE 🔒")
    ciphertext, K_encap = encapsulate(pk)
    print("📦 Clave encapsulada:")
    print(f"🟣 {K_encap.hex()}\n")

    # 3. Desencapsulación de clave
    print_header("🔓 DESENCAPSULACIÓN DE CLAVE 🔓")
    K_decap = decapsulate(sk, ciphertext)
    print("📦 Clave desencapsulada:")
    print(f"🟡 {K_decap.hex()}\n")

    # 4. Comparación de claves
    print_header("📊 COMPARACIÓN DE CLAVES 📊")
    status = "✅ Coinciden" if K_encap == K_decap else "❌ No coinciden"
    print(f"Resultado: {status}\n")



══════════════════════════════════════════════════
           🔑 GENERACIÓN DE CLAVES KEM 🔑           
══════════════════════════════════════════════════

📏 Tamaño clave pública: 24 bytes
📏 Tamaño clave secreta: 16 bytes

📜 Clave Pública (pk):
🟢 (array([[79],
       [44]]), b'\xa2]\xf8\xaa\x98:=\t')

🔐 Clave Secreta (sk):
🟡 b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


══════════════════════════════════════════════════
            🔒 ENCAPSULACIÓN DE CLAVE 🔒            
══════════════════════════════════════════════════

📦 Clave encapsulada:
🟣 8663142850c34f27fd51c6052e507e27


══════════════════════════════════════════════════
          🔓 DESENCAPSULACIÓN DE CLAVE 🔓           
══════════════════════════════════════════════════

📦 Clave desencapsulada:
🟡 8663142850c34f27fd51c6052e507e27


══════════════════════════════════════════════════
            📊 COMPARACIÓN DE CLAVES 📊             
══════════════════════════════════════════════════

Resultado: ✅ Coinciden
